In [0]:
CREATE OR REPLACE TEMPORARY FUNCTION clean_date(d DATE)
RETURNS DATE
RETURN CASE
  WHEN d IS NULL THEN NULL
  WHEN d < DATE '1901-01-01' THEN NULL
  WHEN d IN (
    DATE '1900-01-01', DATE '1901-01-01',
    DATE '1969-12-31', DATE '1970-01-01',
    DATE '2099-12-31', DATE '2999-12-31', DATE '9999-12-31'
  ) THEN NULL
  WHEN d > current_date() + INTERVAL 3 YEARS THEN NULL
  ELSE d
END;

--CREATE OR REPLACE TABLE teamconnect.pca_prism.12_tbl_subinf_other_accounts AS
INSERT OVERWRITE TABLE pca_prism.12_tbl_subinf_other_accounts
SELECT
  s.btn,
  s.dpi_environment,
  array_remove(g.all_btns, s.btn) AS other_btns_same_name_addr
FROM teamconnect.pca_prism.10_tbl_subinf_cur s
JOIN teamconnect.pca_prism.11_tbl_subinf_groups g
  ON s.dpi_environment = g.dpi_environment
 AND lower(s.customer_name)   = g.lk_name
 AND lower(s.service_address) = g.lk_addr;


OPTIMIZE teamconnect.pca_prism.12_tbl_subinf_other_accounts
ZORDER BY (btn, dpi_environment);